1. Read the data into suitable structures.  Display small extracts form the structures as evidence that the data was successfully imported.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sympy import zeros
import plotly.graph_objects as go
import plotly.express as px
import csv

file = open('GPS_and_ANPR.csv', 'r')
csvreader = csv.reader(file, delimiter=',')
occurred_date=[]
vehicle_id = []
location_id = []
exception_code = []
lane = []
weight = []
axle_configuration = []
axles_spacing = []
axle_masses = []



2.	For each site, calculate the total number of times, for all vehicles combined and for each vehicle separately, that a vehicle:
•	passed through the geofence of a WIM scale
•	was seen by an ANPR camera
•	was driving in the WIM lane
•	was successfully weighed by a WIM scale
Display the total counts for all vehicles combined in a table. Display the results per truck in a bar graph.

In [13]:
sites_data = GPS_and_ANPR.groupby(['location_id'], as_index=False)['exception_code'].agg({
    'Through the geofence': lambda group: group.count(), # All exception codes
    'Seen by ANPR Camera': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum(),
    'In WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum(),
    'Weighed': lambda group: ( (group == 2) ).sum()

sites_data_per_truck = GPS_and_ANPR.groupby(['location_id', 'vehicle_id'], as_index=False)['exception_code'].agg({
    'Through the geofence': lambda group: group.count(), # All exception codes
    'Seen by ANPR Camera': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum(),
    'In WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum(),
    'Weighed': lambda group: ( (group == 2) ).sum()

plot = site_data_per_truck.plot.bar(x='vehicle_id')




SyntaxError: invalid syntax. Perhaps you forgot a comma? (2575135786.py, line 5)

3.	Calculate the following performance measures for the total period for all vehicles combined as well as per truck. Display the results for all vehicles combined in a table. Display the results per truck in a bar graph:
a)	The success rate of the ANPR cameras – this is defined as the fraction of cases that a truck, that passed through the WIM geofence based on its GPS data, was also seen by the ANPR camera.
b)	The compliance level of truck drivers in terms of driving in the WIM lane – this is the fraction of cases when the truck was seen by the ANPR camera, and it was also seen driving in the WIM lane.
c)	The compliance level of truck drivers in terms of correctly driving over the WIM scale – this is the fraction of cases when the truck was seen by the ANPR camera, and it was also weighed by the WIM scale.	


In [ ]:

performance_data = GPS_and_ANPR['exception_code'].agg({
    'ANPR cameras success rate': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum() / total_times * 100,
    'Truck drivers compliance level - WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100,
    'Truck drivers compliance level - WIM scale': lambda group: ( (group == 2) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100
})

performance_data_per_truck = GPS_and_ANPR.groupby(['vehicle_id'], as_index=False)['exception_code'].agg({
    'ANPR cameras success rate': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum() / group.count() * 100 ,
    'Truck drivers compliance level - WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100 ,
    'Truck drivers compliance level - WIM scale': lambda group: ( (group == 2) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100 
}) 



,ANPR_Success_rate,WIM_Lane_compliance,WIM_Lane_Correctness
94,16.923982,14.012645,9.101603
95,6.660949,4.974271,4.974271
96,17.968210,13.441603,13.441603


4.	The success rate of the ANPR cameras is influenced by both inaccurate ANPR detections and periods when the system was offline.  To determine the contribution of each factor, calculate the ANPR success rate for all vehicles combined for each site for each day over the period of observations and display the results in a bar chart. 

In [ ]:
GPS_and_ANPR['date'] = GPS_and_ANPR.occurred_date.str[:-6]
GPS_and_ANPR['date'] = pd.to_datetime(GPS_and_ANPR['date'], format='%d/%m/%Y')

ANPR_per_day = GPS_and_ANPR.groupby([ 'location_id', 'date'], as_index=False)['exception_code'].agg({
    'ANPR cameras success rate': lambda group: ( ( (group == 1) | (group == 2) | (group == 3) ).sum() / group.count() * 100 ) if group.count() else None
})

ANPR_per_day = ANPR_per_day.sort_values(by=['location_id','date'])



,Date,Total_vehicles,Detected_vehicles,Success_rate
1,2021-08-17,39,4,10.256410
2,2021-08-18,27,6,22.222222
3,2021-08-19,28,4,14.285714
4,2021-08-20,18,2,11.111111
5,2021-08-21,19,2,10.526316
...,...,...,...,...
233,2022-05-02,33,19,57.575758
234,2022-05-03,27,18,66.666667
235,2022-05-04,25,9,36.000000
236,2022-05-05,43,19,44.186047


,Date,Total_vehicles,Detected_vehicles,Success_rate
1,2021-08-17,12,0,0.000000
2,2021-08-18,22,0,0.000000
3,2021-08-19,11,0,0.000000
4,2021-08-20,3,0,0.000000
5,2021-08-21,5,0,0.000000
...,...,...,...,...
226,2022-05-02,21,14,66.666667
227,2022-05-03,20,12,60.000000
228,2022-05-04,14,5,35.714286
229,2022-05-05,26,13,50.000000


,Date,Total_vehicles,Detected_vehicles,Success_rate
1,2021-08-16,3,0,0.000000
2,2021-08-17,10,2,20.000000
3,2021-08-18,13,6,46.153846
4,2021-08-19,22,5,22.727273
5,2021-08-20,8,0,0.000000
...,...,...,...,...
221,2022-05-02,11,8,72.727273
222,2022-05-03,7,6,85.714286
223,2022-05-04,15,10,66.666667
224,2022-05-05,18,10,55.555556


5.	Generate histograms for the daily ANPR success rate for each site for all trucks.  Use 25 bins for the histograms.  Display graphs of the histograms, and comment on the modes of behaviour that are observed.  

In [ ]:
ANPR_per_day_hist = ANPR_per_day.plot.hist( 
    column=["ANPR cameras success rate"], by='location_id', 
    bins=25, edgecolor='white', linewidth=0.5, figsize=(10, 6),
    title='ANPR Camera Daily Success Rate Frequency',
)

6.	Sort the available data based on date and time for each site.  Determine when no ANPR transactions were recorded for a period of at least 6 hours.  Use this as condition to determine if the ANPR system at each site was working at that time.  For observations that occurred when the system was not working the exception code must be changed from 0 to 6.	

In [ ]:

if(group.exception_code.iloc[i] == 0)
t1 = group.occurred_date.iloc[start]
t2 = group.occurred_date.iloc[end]

# Difference in seconds
difference_sec = pd.Timedelta(t2 - t1).total_seconds() 
difference_hrs = difference_sec/3600
if(difference_hrs >= 6):
    # Add to total time offline
    time_offline += difference_hrs    

start_name = group.index[start]
    end_name = group.index[end]

    # loc takes row condition, col condition 

    group.loc[start_name:end_name, 'exception_code'] = 6



7.	For each site, fraction of total time when the ANPR camera system appeared to be offline.

In [ ]:
total_time_sec = pd.Timedelta(group.occurred_date.iloc[-1] - group.occurred_date.iloc[0]).total_seconds() 
total_time_hrs = total_time_sec/3600
print('Total time:', total_time_hrs)

fraction = time_offline/total_time_hrs




8.	Repeat the calculation of ANPR success rate and driver compliance for all vehicles combined as well as per truck, by only using data for the periods when the ANPR system was working and compare it against the results obtained for all time. Comment on the most important causes for unsatisfactory ANPR performance.	

In [ ]:

performance_data = GPS_and_ANPR['exception_code'].agg({
    'ANPR cameras success rate': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum() / total_times * 100,
    'Truck drivers compliance level - WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100,
    'Truck drivers compliance level - WIM scale': lambda group: ( (group == 2) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100
})

performance_data_per_truck = GPS_and_ANPR.groupby(['vehicle_id'], as_index=False)['exception_code'].agg({
    'ANPR cameras success rate': lambda group: ( (group == 1) | (group == 2) | (group == 3) ).sum() / group.count() * 100 ,
    'Truck drivers compliance level - WIM lane': lambda group: ( (group == 2) | (group == 3) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100 ,
    'Truck drivers compliance level - WIM scale': lambda group: ( (group == 2) ).sum() / ( (group == 1) | (group == 2) | (group == 3) ).sum() * 100 
}) 



,ANPR_Success_rate,WIM_Lane_compliance,WIM_Lane_Correctness
94,18.987133,15.720884,10.211151
95,18.034056,13.467492,13.467492
96,39.513678,29.559271,29.559271
